I wrote this code a while back, and I don't remember everything about where it was used. I may have used it to help create an SQL table of loans fuzzy matched to listings with duplicates removed; howevever, I may have abandoned this script in favor of another method. 

In [1]:
import sqlalchemy as sql
import pandas as pd
import logging

In [2]:
log = logging.getLogger()
log.setLevel('INFO')

In [3]:
#db connection
creds = 'username:password'   #fill in
eng   = sql.create_engine('postgresql://%s@localhost:5432/p2p_lending' %creds)
eng   = eng.connect()

In [4]:
#loans prep
loan_cols = ('loan_number,origination_date,amount_borrowed,borrower_rate,' +
             'prosper_rating,term')
#stmt = 'select %s from loans' %loan_cols
stmt = 'select * from loans'
loans = pd.read_sql(stmt, con=eng)

#listings prep
stmt2 = ("select listing_number from listings where " +
         "loan_origination_date='%s' and amount_funded='%.2f' " +
         "and borrower_rate='%.4f' and prosper_rating='%s' and listing_term='%d'")

In [ ]:
#make loans_nodups
for row in loans.iterrows():
    #query listings
    row = row[1]    #output only (no iterator)
    vals = (row.origination_date.strftime('%Y-%m-%d'),
            row.amount_borrowed, 
            row.borrower_rate,
            row.prosper_rating,
            row.term)
    match_listings = pd.read_sql(stmt2 %vals, con=eng)
    if len(match_listings.index) == 1:
        #write unique records to loans_nodups
        row['listing_number'] = match_listings.listing_number.values[0]
        row = row.to_frame().T    #change row from Series to dataframe
        row.to_sql('loans_nodups', con=eng, if_exists='append', index=False)
    if len(match_listings.index) == 0:
        logging.warn('no match for row: %s' %str(tuple(row)))

In [63]:
row.T

,loan_number,origination_date,amount_borrowed,borrower_rate,prosper_rating,term,listing_number
0,123672,2014-01-02,25000,0.1765,C,60,1074609


In [27]:
date = onerow.origination_date.values[0]

In [28]:
date.strftime('%Y-%m-%d')

'2014-01-02'

In [12]:
loans_ex = pd.read_sql('select * from loans limit 10', con=eng)

In [ ]:
onerow = loans_ex.head(1).copy()
onerow['listing_number'] = 1111

In [20]:
onerow.to_sql(name='loans_nodups', con=eng, if_exists='append', index=False)

In [3]:
ex = eng.execute('select * from loans limit 2')

In [4]:
fst = ex.fetchone()

scd = ex.fetchone()

In [6]:
fst

(123672, 25000.0, 0.1765, u'C', 60, 33, datetime.date(2014, 1, 2), 0, 13949.89, -551.99, 11050.1, 9730.781, 0.0, 0.0, 0.0, 1, u'CURRENT', None, None, datetime.date(2016, 11, 2), 630.09, 630.08531828492, 0.83123524)

In [9]:
cur = eng.execute('insert into loans_nodups values %s' %(str(fst).replace(')',',1111)')))

ProgrammingError: (psycopg2.ProgrammingError) relation "loans_dedups" does not exist
LINE 1: insert into loans_dedups values (123672, 25000.0, 0.1765, u'...
                    ^
 [SQL: "insert into loans_dedups values (123672, 25000.0, 0.1765, u'C', 60, 33, datetime.date(2014, 1, 2,1111), 0, 13949.89, -551.99, 11050.1, 9730.781, 0.0, 0.0, 0.0, 1, u'CURRENT', None, None, datetime.date(2016, 11, 2,1111), 630.09, 630.08531828492, 0.83123524,1111)"]